In [ ]:
%%capture
import numpy as np
!pip install qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, assemble
from qiskit import execute
from qiskit import BasicAer as Aer
from qiskit.tools.visualization import circuit_drawer, plot_histogram

## Hamiltonian and mapping

The following fermionic model describes the transition to a superconducting phase. We will consider a single particle space $\mathcal{H}$ with $2\Omega$ modes composed by $\Omega$ orthogonal single particle states $k$ and the corresponding $\Omega$ time-reversed states $\bar{k}$ described by the following Hamiltonian:

$H = \sum_k \varepsilon_k (c_k^\dagger c_k + c_{\bar{k}}^\dagger c_{\bar{k}}) - \sum_{kk'} G_{k k'}c_{k'}^\dagger c_{\bar{k'}}^\dagger c_{\bar{k}} c_k$


with $\varepsilon_k$ the energy of the level $k$ and $\bar{k}$. We will consider the half filled case, i.e. a fixed number of fermions $N=\Omega$. We will look at the case of equally spaced spectrum $\varepsilon_{k+1}-\varepsilon_k = \varepsilon\;\; \forall k $ and $G_{kk'} = G\geq 0 \;\; \forall k, k'$

En el estado fundamental, el estado no tiene pares rotos, es decir los modos $k$ y $\bar{k}$ están simultaneamente ocupados. Esto permite realizar el mapeo  de ocupación de niveles a spines (en lugar de ocupación de modos a spines, que requiría el doble de spines). En este caso vamos a tomar $\Omega=4$ y lo vamos a mapear a un sistema de 4 spines. Para entender el mapeo, a modo de ejemplo si tuviesemos 2 niveles, tendríamos que ir de  la base $\{00,01,10,11\}$ a la base $\{\downarrow \downarrow ,\downarrow \uparrow,\uparrow\downarrow ,\uparrow\uparrow\}$.  El estado $|\psi\rangle_f = \frac{1}{\sqrt{2}}(c_1^\dagger + c_2^\dagger)|0\rangle = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle) \rightarrow |\psi\rangle_s =\frac{1}{\sqrt{2}}(|\downarrow \uparrow \rangle + |\uparrow\downarrow\rangle ) $. Desde ya que el tener numero fijo de fermiones haría que solo algunos de estos estados se prendan. 

La fermionicidad se perdería en principio al fijar la base (necesario para mapear a índices de qubits), pero al mapear con ocupaciones de niveles estamos tomando la base privilegiada en fermiones (que diagonaliza el Hamiltoniano). Estamos fijando la base privilegiada y por ende no estamos perdiendo la fermionicidad. Si hubiesemos mapeado modos a spines independientemente de la base que diagonaliza el hamiltoniano, como se haría en Jordan Wigner convencional, hubiesemos perdido fermionicidad realmente.

## Dataset
Let's download the dataset


In [ ]:
import requests
url = 'https://raw.githubusercontent.com/Marco-Di-Tullio/Fermionic-dataset/main/estados_super.csv'
r = requests.get(url, allow_redirects=True)
open('estados_super.csv', 'wb').write(r.content);


This is how it looks:

In [ ]:
import pandas as pd
df = pd.read_csv('estados_super.csv')
df.head()

,g,0000,0001,0010,0011,0100,0101,0110,0111,1000,1001,1010,1011,1100,1101,1110,1111,label
0,0.00,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,1.000000,0.0,0.0,0.0,0.0
1,0.01,0.0,0.0,0.0,0.000015,0.0,0.001675,0.002517,0.0,0.0,0.002517,0.005025,0.0,0.999980,0.0,0.0,0.0,0.0
2,0.02,0.0,0.0,0.0,0.000059,0.0,0.003367,0.005067,0.0,0.0,0.005067,0.010099,0.0,0.999918,0.0,0.0,0.0,0.0
3,0.03,0.0,0.0,0.0,0.000133,0.0,0.005075,0.007650,0.0,0.0,0.007650,0.015221,0.0,0.999813,0.0,0.0,0.0,0.0
4,0.04,0.0,0.0,0.0,0.000239,0.0,0.006801,0.010266,0.0,0.0,0.010266,0.020389,0.0,0.999664,0.0,0.0,0.0,0.0


In [ ]:
datos = df.to_numpy()
datos

array([[0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.01, 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.02, 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       ...,
       [4.98, 0.  , 0.  , ..., 0.  , 0.  , 1.  ],
       [4.99, 0.  , 0.  , ..., 0.  , 0.  , 1.  ],
       [5.  , 0.  , 0.  , ..., 0.  , 0.  , 1.  ]])

## Initializing Qiskit

El comando clave es initialize(). Uno le dá un vector, y el programa entiende las operaciones para generarlo. La teoría detrás de eso está explicada en [este tutorial](https://github.com/Qiskit/qiskit-tutorials/blob/master/tutorials/circuits/3_summary_of_quantum_operations.ipynb). 
Los estados de input viven en dimensión $2^n$, lo que facilita escribir estados generales entrelazados. 

In [ ]:
# I remove labels and couplings
estado = datos[:,1:-1]

# Number of state
i = 25

# I convert it into a list for inputing qiskit
state=estado[i,:].tolist()
label = int(datos[i,-1])

# Desired vector lives in te 2^n space
desired_vector = state

q = QuantumRegister(4)
c = ClassicalRegister(1)

qc = QuantumCircuit(q,c)

qc.initialize(desired_vector, [q[0],q[1],q[2],q[3]])
qc.draw()
# Aparece en 2 lineas porque no entra

»
q42_0: »
       »
q42_1: »
       »
q42_2: »
       »
q42_3: »
       »
 c6: 1/»
       »
«       ┌────────────────────────────────────────────────────────────────────────────────────┐
«q42_0: ┤0                                                                                   ├
«       │                                                                                    │
«q42_1: ┤1                                                                                   ├
«       │  initialize(0,0,0,0.010133,0,0.04679,0.072109,0,0,0.072109,0.13663,0,0.9842,0,0,0) │
«q42_2: ┤2                                                                                   ├
«       │                                                                                    │
«q42_3: ┤3                                                                                   ├
«       └────────────────────────────────────────────────────────────────────────────────────┘
« c6: 1/══════════════════════════════════════════════════════════════════════════════════════
«

In [ ]:
backend = Aer.get_backend('statevector_simulator')
job = execute(qc, backend)
qc_state = job.result().get_statevector(qc)
# The resulting state is written in the 2^n basis
qc_state

array([0.        +0.j, 0.        +0.j, 0.        +0.j, 0.01013337+0.j,
       0.        +0.j, 0.04679042+0.j, 0.07210914+0.j, 0.        +0.j,
       0.        +0.j, 0.07210914+0.j, 0.13663267+0.j, 0.        +0.j,
       0.98419512+0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j])

In [ ]:
# qc.measure(q[0], c[0])
# job = execute(qc, backend, shots=100)
# result = job.result()
# result.get_counts(qc)

## Train-test splitting for Machine Learning

In [ ]:
from sklearn.model_selection import train_test_split

# I remove  couplings
estados = datos[:,1:]
train, test = train_test_split(estados, test_size=0.2, random_state=0)



In [ ]:
train[1,:]

array([0.        , 0.        , 0.        , 0.07529731, 0.        ,
       0.14444913, 0.21527301, 0.        , 0.        , 0.21527301,
       0.35389142, 0.        , 0.86921844, 0.        , 0.        ,
       0.        , 1.        ])